In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import vgg16
from torchvision import datasets, transforms
import copy
import time

In [3]:
# 1. Tải model VGG-16 và in ra thông tin mô hình
model = vgg16(pretrained=True)
print("Cấu trúc mô hình VGG-16:")
print(model)


d:\Study For Better Future\BCU\Research - ML\Week 6\VGG-16\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Study For Better Future\BCU\Research - ML\Week 6\VGG-16\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Cấu trúc mô hình VGG-16:
VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride

In [4]:
# Hàm để tính số lượng tham số của mô hình
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Số lượng tham số của mô hình ban đầu: {count_parameters(model)}")

Số lượng tham số của mô hình ban đầu: 138357544


In [5]:
# 2. Thực hiện prune một filter và xem sự thay đổi về shape
def prune_filter(model, layer_index, filter_index):
    conv_layer = model.features[layer_index]

    # Tạo một lớp tích chập mới với số filter giảm đi 1
    new_conv = nn.Conv2d(in_channels=conv_layer.in_channels,
                         out_channels=conv_layer.out_channels - 1,
                         kernel_size=conv_layer.kernel_size,
                         stride=conv_layer.stride,
                         padding=conv_layer.padding,
                         dilation=conv_layer.dilation,
                         groups=conv_layer.groups,
                         bias=conv_layer.bias is not None)

    # Sao chép trọng số và bias, ngoại trừ filter được prune
    new_filters = torch.cat((conv_layer.weight.data[:filter_index], conv_layer.weight.data[filter_index+1:]))
    new_conv.weight.data = new_filters

    if conv_layer.bias is not None:
        new_biases = torch.cat((conv_layer.bias.data[:filter_index], conv_layer.bias.data[filter_index+1:]))
        new_conv.bias.data = new_biases

    # Thay thế lớp tích chập cũ bằng lớp mới trong mô hình
    model.features[layer_index] = new_conv

    return model

In [6]:
# Prune filter đầu tiên của lớp tích chập đầu tiên
pruned_model = prune_filter(copy.deepcopy(model), 0, 0)

print("\nShape của lớp tích chập đầu tiên trước khi pruning:")
print(model.features[0].weight.shape)
print("Shape của lớp tích chập đầu tiên sau khi pruning:")
print(pruned_model.features[0].weight.shape)

print(f"\nSố lượng tham số sau khi pruning: {count_parameters(pruned_model)}")


Shape của lớp tích chập đầu tiên trước khi pruning:
torch.Size([64, 3, 3, 3])
Shape của lớp tích chập đầu tiên sau khi pruning:
torch.Size([63, 3, 3, 3])

Số lượng tham số sau khi pruning: 138357516


In [7]:
# Chuẩn bị dữ liệu
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [8]:
# Sử dụng một tập dữ liệu nhỏ để demo (1000 ảnh)
dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
subset_size = 1000
subset_indices = torch.randperm(len(dataset))[:subset_size]
subset = torch.utils.data.Subset(dataset, subset_indices)
dataloader = torch.utils.data.DataLoader(subset, batch_size=32, shuffle=True)

100.0%


Extracting ./data\cifar-10-python.tar.gz to ./data


In [9]:
# Hàm train
def train_model(model, dataloader, criterion, optimizer, num_epochs=5):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        model.train()
        running_loss = 0.0

        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

 # Check the input channels and adjust the first convolutional layer accordingly
            input_channels = inputs.shape[1]
            if input_channels != model.features[0].in_channels:
                # If input channels don't match, create a new Conv2d layer
                new_conv1 = nn.Conv2d(input_channels, model.features[0].out_channels, kernel_size=model.features[0].kernel_size, stride=model.features[0].stride, padding=model.features[0].padding)

                # Replace the first layer in the model's features
                model.features[0] = new_conv1
                model.to(device) # Move model back to device if necessary


            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(dataloader.dataset)
        print(f'Training Loss: {epoch_loss:.4f}')

    return model, epoch_loss

In [10]:
# Train mô hình gốc
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

print("\nTraining mô hình gốc")
start_time = time.time()
_, original_loss = train_model(model, dataloader, criterion, optimizer)
end_time = time.time()
print(f"Thời gian training mô hình gốc: {end_time - start_time:.2f} giây")
print(f"Loss cuối cùng của mô hình gốc: {original_loss:.4f}")


Training mô hình gốc
Epoch 1/5
----------


KeyboardInterrupt: 

In [ ]:
# Train mô hình đã pruning
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(pruned_model.parameters(), lr=0.001, momentum=0.9)

print("\nTraining mô hình đã pruning")
start_time = time.time()
_, pruned_loss = train_model(pruned_model, dataloader, criterion, optimizer)
end_time = time.time()
print(f"Thời gian training mô hình đã pruning: {end_time - start_time:.2f} giây")
print(f"Loss cuối cùng của mô hình đã pruning: {pruned_loss:.4f}")

print(f"\nChênh lệch loss: {abs(original_loss - pruned_loss):.4f}")


Training mô hình đã pruning...
Epoch 1/5
----------


RuntimeError: Given groups=1, weight of size [64, 64, 3, 3], expected input[32, 63, 224, 224] to have 64 channels, but got 63 channels instead